In [1]:
import os
import sys
import pandas as pd
import numpy as np
import glob

In [2]:
results_df = pd.read_csv("tests/evaluation_19_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20240204.tsv", sep='\t')

In [3]:
results_df

,query_id,query,passage_id,mT5,splade,colbertx,bm25_e5,bm25,e5,e5_base,...,bm25_e5_1M_massiveweb,text-embbeding-3-large_1M_massiveweb,text-embbeding-3-small_1M_massiveweb,passage,score,reasoning,usage,cost,duration,saved_cost
0,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-82-13187_0,False,False,True,False,False,False,False,...,False,False,False,Mamíferos do Brasil - Zoologia | Museu Naciona...,1,A passagem lista vários mamíferos da fauna bra...,"{'prompt_tokens': 765, 'completion_tokens': 75...",0.00000,6.693229,0.02745
1,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-59-04599_2,False,False,True,False,False,False,False,...,False,False,False,Segundo uma lista levantada pelo Comitê Brasil...,1,A passagem descreve várias espécies de aves en...,"{'prompt_tokens': 860, 'completion_tokens': 82...",0.00000,5.163547,0.03072
2,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-64-06285_39,True,True,False,False,False,False,True,...,False,False,False,"A exuberante fauna e flora brasileiras, relata...",3,A passagem menciona que o Brasil tem a maior b...,"{'prompt_tokens': 840, 'completion_tokens': 46...",0.00000,5.185688,0.02796
3,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-77-06884_1,False,True,True,False,False,False,False,...,False,False,False,O tamanho do Brasil e a grande variedade de cl...,1,A passagem menciona a alta biodiversidade do B...,"{'prompt_tokens': 812, 'completion_tokens': 81...",0.00000,5.882017,0.02922
4,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-78-09747_0,True,False,False,False,True,False,False,...,False,False,False,Onça-pintada - Escola Kids Qual matéria está p...,1,A passagem menciona que a onça-pintada é um do...,"{'prompt_tokens': 805, 'completion_tokens': 96...",0.00000,5.774773,0.02991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4839,199,Quais foram os principais artistas do Impressi...,clueweb22-pt0001-29-09969_2,False,False,False,False,False,False,False,...,True,False,False,Considerado o maior pintor do romantismo franc...,2,A passagem menciona Edgar Degas como um dos fu...,"{'prompt_tokens': 840, 'completion_tokens': 86...",0.03036,5.756289,0.00000
4840,199,Quais foram os principais artistas do Impressi...,clueweb22-pt0000-26-12708_2,False,False,False,False,False,False,False,...,True,False,False,"O pintor mais conhecido é Van Gogh, conhecido ...",2,A passagem menciona Monet como o impressionist...,"{'prompt_tokens': 750, 'completion_tokens': 73...",0.02688,4.359926,0.00000
4841,199,Quais foram os principais artistas do Impressi...,clueweb22-pt0000-17-07130_2,False,False,False,False,False,False,False,...,True,False,False,"Dadaísmo: corrente mais radical, mostra-se tot...",0,A passagem não menciona o Impressionismo nem s...,"{'prompt_tokens': 747, 'completion_tokens': 59...",0.02595,3.649166,0.00000
4842,199,Quais foram os principais artistas do Impressi...,clueweb22-pt0001-22-00434_5,False,False,False,False,False,False,False,...,True,False,False,"Em face de tal refutação, coloca-se não apenas...",1,A passagem menciona a tradição do impressionis...,"{'prompt_tokens': 817, 'completion_tokens': 12...",0.03201,3.651713,0.00000


In [5]:
score_stats = np.unique(results_df['score'].to_numpy(), return_counts=True)

In [6]:
score_stats

(array([0, 1, 2, 3]), array([2462,  974,  752,  656]))

In [7]:
score_stats[1] / score_stats[1].sum()

array([0.50825764, 0.20107349, 0.1552436 , 0.13542527])

In [26]:
score_stats[1][1:].sum()

2382

In [12]:
results_df.columns

Index(['query_id', 'query', 'passage_id', 'mT5', 'splade', 'colbertx',
       'bm25_e5', 'bm25', 'e5', 'e5_base', 'splade_mT5', 'bm25_1M',
       'bm25_1M_massiveweb', 'colbertx_1M_massiveweb', 'e5_colbertx_rrf',
       'e5_splade_rrf', 'e5_splade_rrf_fix', 'adav2', 'bm25_mT5_1M_massiveweb',
       'bm25_mT5_1M_massiveweb_fix', 'bm25_e5_1M_massiveweb',
       'text-embbeding-3-large_1M_massiveweb',
       'text-embbeding-3-small_1M_massiveweb', 'passage', 'score', 'reasoning',
       'usage', 'cost', 'duration', 'saved_cost'],
      dtype='object')

In [13]:
HOW_MANY_RESULTS=23

In [14]:
np.unique(results_df.iloc[:, 3:HOW_MANY_RESULTS].sum(axis=1), return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19]),
 array([3077,  710,  335,  225,  150,   99,   70,   52,   54,   24,   21,
           5,    7,    1,    6,    4,    1,    2,    1]))

In [15]:
single_system_df = results_df.loc[results_df.iloc[:, 3:HOW_MANY_RESULTS].sum(axis=1) == 1]

In [18]:
single_score_stats = np.unique(single_system_df['score'].to_numpy(), return_counts=True)

In [19]:
single_score_stats

(array([0, 1, 2, 3]), array([1870,  596,  382,  229]))

In [20]:
single_score_stats[1] / single_score_stats[1].sum()

array([0.60773481, 0.19369516, 0.1241469 , 0.07442314])

In [33]:
single_score_stats[1][1:]

array([596, 382, 229])

In [27]:
single_score_stats[1][1:].sum()

1207

In [44]:
score_stats_table = []

for which_score in score_stats[0]:
    score_stats_table.append({"score": which_score, 
                              "all query-passages": score_stats[1][which_score],
                              "single-system query-passages": single_score_stats[1][which_score],
                              "single-system / all": single_score_stats[1][which_score] / score_stats[1][which_score]})

In [46]:
score_stats_table.append({"score": "Minimally relevant", 
                          "all query-passages": score_stats[1][1:].sum(), 
                          "single-system query-passages": single_score_stats[1][1:].sum(), 
                          "single-system / all": single_score_stats[1][1:].sum() / score_stats[1][1:].sum()})

In [50]:
score_stats_table.append({"score": "Total", 
                          "all query-passages": score_stats[1].sum(), 
                          "single-system query-passages": single_score_stats[1].sum(), 
                          "single-system / all": single_score_stats[1].sum() / score_stats[1].sum()})

In [51]:
single_score_stats[1][1:].sum() / score_stats[1][1:].sum()

0.5067170445004198

In [52]:
pd.DataFrame(score_stats_table)

,score,all query-passages,single-system query-passages,single-system / all
0,0,2462,1870,0.759545
1,1,974,596,0.61191
2,2,752,382,0.507979
3,3,656,229,0.349085
4,Minimally relevant,2382,1207,0.506717
5,Total,4844,3077,
6,Total,4844,3077,0.635219


In [21]:
single_system_df.iloc[:, 3:HOW_MANY_RESULTS].sum()

mT5                                      60
splade                                  151
colbertx                                115
bm25_e5                                 115
bm25                                    248
e5                                      122
e5_base                                 262
splade_mT5                               86
bm25_1M                                 362
bm25_1M_massiveweb                      255
colbertx_1M_massiveweb                  199
e5_colbertx_rrf                          32
e5_splade_rrf                           176
e5_splade_rrf_fix                        29
adav2                                   141
bm25_mT5_1M_massiveweb                  278
bm25_mT5_1M_massiveweb_fix               94
bm25_e5_1M_massiveweb                   123
text-embbeding-3-large_1M_massiveweb    124
text-embbeding-3-small_1M_massiveweb    105
dtype: int64

In [23]:
(single_system_df.iloc[:, 3:HOW_MANY_RESULTS].sum() / 500)

mT5                                     0.120
splade                                  0.302
colbertx                                0.230
bm25_e5                                 0.230
bm25                                    0.496
e5                                      0.244
e5_base                                 0.524
splade_mT5                              0.172
bm25_1M                                 0.724
bm25_1M_massiveweb                      0.510
colbertx_1M_massiveweb                  0.398
e5_colbertx_rrf                         0.064
e5_splade_rrf                           0.352
e5_splade_rrf_fix                       0.058
adav2                                   0.282
bm25_mT5_1M_massiveweb                  0.556
bm25_mT5_1M_massiveweb_fix              0.188
bm25_e5_1M_massiveweb                   0.246
text-embbeding-3-large_1M_massiveweb    0.248
text-embbeding-3-small_1M_massiveweb    0.210
dtype: float64

In [24]:
(single_system_df.iloc[:, 3:HOW_MANY_RESULTS].sum() / 500).mean()

0.3077